In [97]:
import numpy as np
import pandas as pd
from collections import Counter

In [83]:
queries = pd.read_csv('queries.csv', delimiter="\t", names=["query_id", "query"]).iloc[1:]
queries = queries.dropna()
queries.head(10)

,query_id,query
1,1,what similarity laws must be obeyed when const...
2,2,what are the structural and aeroelastic proble...
3,4,what problems of heat conduction in composite ...
4,8,can a criterion be developed to show empirical...
5,9,what chemical kinetic system is applicable to ...
6,10,what theoretical and experimental guides do we...
7,12,is it possible to relate the available pressur...
8,13,what methods -dash exact or approximate -dash ...
9,15,papers on internal /slip flow/ heat transfer s...
10,18,are real-gas transport properties for air avai...


In [31]:
train = pd.read_csv('relevance_train.csv', delimiter="\t", names=["query_id", "doc_id", "relevance"]).iloc[1:]
train.head(5)

,query_id,doc_id,relevance
1,1,184,2
2,1,29,2
3,1,31,2
4,1,12,3
5,1,51,3


In [34]:
test = pd.read_csv('relevance_test.csv', delimiter=",", names=["query_id", "doc_id"]).iloc[1:]
test.head(10)

,query_id,doc_id
1,126,974
2,126,1326
3,126,187
4,126,969
5,126,970
6,126,971
7,126,972
8,126,973
9,126,942
10,127,101


In [47]:
print queries.shape
print train.shape
print test.shape

(226, 2)
(990, 3)
(847, 2)


# Состав документов

In [41]:
train.describe()

,query_id,doc_id,relevance
count,990,990,989
unique,125,611,5
top,23,629,3
freq,33,9,377


In [123]:
print set(train.relevance)
print Counter(train.doc_id).most_common(5)
print Counter(test.doc_id).most_common()

set([nan, '1', '3', '2', '-1', '4'])
[('629', 9), ('24', 6), ('798', 6), ('570', 6), ('283', 6)]
[('1062', 5), ('743', 5), ('1304', 5), ('1305', 5), ('889', 5), ('1213', 5), ('843', 5), ('951', 5), ('1018', 4), ('1016', 4), ('1173', 4), ('1045', 4), ('666', 4), ('769', 4), ('1107', 4), ('763', 4), ('1074', 4), ('1075', 4), ('1188', 4), ('950', 4), ('954', 4), ('1258', 3), ('1019', 3), ('1017', 3), ('1012', 3), ('559', 3), ('858', 3), ('93', 3), ('1052', 3), ('1285', 3), ('948', 3), ('1191', 3), ('1300', 3), ('459', 3), ('1309', 3), ('1379', 3), ('1008', 3), ('458', 3), ('655', 3), ('15', 3), ('887', 3), ('888', 3), ('1043', 3), ('1042', 3), ('1044', 3), ('667', 3), ('1385', 3), ('1386', 3), ('1383', 3), ('838', 3), ('839', 3), ('572', 3), ('603', 3), ('630', 3), ('161', 3), ('1104', 3), ('891', 3), ('894', 3), ('285', 3), ('982', 3), ('983', 3), ('111', 3), ('421', 3), ('423', 3), ('1392', 3), ('1391', 3), ('841', 3), ('840', 3), ('1024', 3), ('1020', 3), ('604', 3), ('155', 3), ('602'

611 разных документов, 226 запросов

# Позапросные фичи

Описание факторов:

f1 - количество слов в запросе
f2 - количество букв в запросе
f3 - средняя длина слова
f4 - 
f5 - 

In [78]:
def f1(query):
    return len(query.split(" "))

def f2(query):
    return sum(len(word) for word in query.split(" "))

def f3(query):
    return np.mean([len(word) for word in query.split(" ")])

def f4(query):
    return 

In [72]:
queries["f1"] = queries["query"].map(f1)

In [74]:
queries["f2"] = queries["query"].map(f2)

In [79]:
queries["f3"] = queries["query"].map(f3)

In [80]:
queries.head()

,query_id,query,f1,f2,f3
1,1,what similarity laws must be obeyed when const...,17,89,5.235294
2,2,what are the structural and aeroelastic proble...,16,82,5.125000
3,4,what problems of heat conduction in composite ...,15,64,4.266667
4,8,can a criterion be developed to show empirical...,30,170,5.666667
5,9,what chemical kinetic system is applicable to ...,12,69,5.750000


# Подкументные факторы

In [90]:
df = pd.merge(queries, train, on="query_id")
df.head()

,query_id,query,doc_id,relevance
0,1,what similarity laws must be obeyed when const...,184,2
1,1,what similarity laws must be obeyed when const...,29,2
2,1,what similarity laws must be obeyed when const...,31,2
3,1,what similarity laws must be obeyed when const...,12,3
4,1,what similarity laws must be obeyed when const...,51,3


In [ ]:
for d in set(train["doc_id"]):
    print d, Counter((" ".join(df[df.doc_id == d]["query"].ravel())).split(" ")).most_common(6)

### В итоге все это не используется и работает классификатор на одной фиче:

In [198]:
def get_query_by_qid(qid):
    l = np.ravel(queries[queries.query_id == str(qid)]["query"])[0]
    return l[0] if l else None

In [205]:
def get_queries_by_docid(docid):
    l = np.ravel(df[df.doc_id == str(docid)]["query"])
    return l

In [207]:
def get_docs_by_query(qid):
    l = np.ravel(df[df.query_id == str(qid)]["doc_id"])
    return l

In [215]:
x = [[1, 2], [2, 3], [3, 1], [4, 1.2]]


[[3, 1], [4, 1.2], [1, 2], [2, 3]]

In [220]:
for q in sorted(set(test.query_id.astype(int))):
    docs = list(test[test.query_id == str(q)]["doc_id"])
    
    res = []
    for d in docs:
        res.append([d, len(get_queries_by_docid(d)) - len(get_docs_by_query(q))])
                   
    res = sorted(res, key=lambda i: i[1])
                   
    for d in res:
        print "{},{}".format(q, d[0])

126,974
126,1326
126,187
126,969
126,970
126,971
126,972
126,973
126,942
127,164
127,981
127,982
127,983
127,944
127,101
128,985
128,990
128,945
129,985
129,987
129,984
129,988
129,989
129,986
129,990
129,945
130,1008
130,948
130,766
130,858
130,859
130,12
131,1014
131,1020
131,1013
131,1016
131,1017
131,1012
131,1018
131,1019
131,950
132,1014
132,1013
132,1015
132,1016
132,1012
132,1018
132,1019
132,1017
132,951
132,1023
132,1020
132,952
132,1024
132,1025
132,1026
132,950
133,1020
133,1018
133,1019
133,1024
133,1012
133,1016
133,1022
133,950
134,1027
134,1028
134,951
135,1018
135,1017
135,1023
135,1024
135,1025
135,1026
135,950
135,1016
135,951
136,1021
136,1022
136,1034
136,951
137,1029
137,1034
137,951
137,1021
137,952
138,1036
138,953
138,846
139,1031
139,1032
139,1035
139,1037
139,953
140,1038
140,1039
140,1041
140,1042
140,1043
140,1044
140,954
141,1038
141,1041
141,1043
141,1044
141,1039
141,1042
141,954
142,1042
142,954
143,1043
143,1044
143,954
144,840
144,1045
144,763
144,838